# StealthGAN-IDS Training on Google Colab

This notebook provides a complete pipeline for training and evaluating StealthGAN-IDS on Google Colab.

**Features:**
- Automatic GPU detection and setup
- Dataset download and preprocessing
- Training with checkpointing
- Evaluation and visualization
- Easy result download

**Supported Datasets:**
- NSL-KDD (legacy)
- CIC-IDS2017
- CIC-IDS2018
- UNSW-NB15

## 1. Setup and Installation

In [ ]:
# Check GPU availability
import torch
import os
from pathlib import Path

print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  No GPU detected! Training will be very slow on CPU.")
    print("Go to Runtime > Change runtime type > Hardware accelerator > GPU")

In [ ]:
# Clone the repository
# Option 1: Clone from GitHub (update with your repo URL)
repo_url = "https://github.com/yourusername/SGAN-IDS.git"  # ⚠️ UPDATE THIS
repo_dir = Path("/content/SGAN-IDS")

# Option 2: Upload repository as zip and extract
# 1. Zip your repository locally
# 2. Upload via Colab: Files > Upload
# 3. Uncomment and run:
# !unzip -q /content/SGAN-IDS.zip -d /content/

if repo_dir.exists():
    print(f"Repository already exists at {repo_dir}")
    os.chdir(repo_dir)
    # Try to pull updates if it's a git repo
    if (repo_dir / ".git").exists():
        !git pull
else:
    if repo_url != "https://github.com/yourusername/SGAN-IDS.git":
        !git clone {repo_url} {repo_dir}
        os.chdir(repo_dir)
    else:
        print("⚠️  Please update repo_url with your repository URL")
        print("Or upload your repository as a zip file")
        print("\nAlternative: Upload SGAN-IDS.zip and run:")
        print("  !unzip -q /content/SGAN-IDS.zip -d /content/")
        raise RuntimeError("Repository not configured - update repo_url or upload zip")

print(f"Working directory: {os.getcwd()}")

In [ ]:
# Install dependencies
# Check if torch is already installed with CUDA support (Colab has it pre-installed)
if not torch.cuda.is_available():
    print("Installing PyTorch with CUDA support...")
    !pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu118
else:
    print(f"PyTorch {torch.__version__} already installed with CUDA support ✓")

!pip install -q -r requirements.txt

# Optional: Install additional evaluation dependencies
!pip install -q xgboost lightgbm pyyaml

print("✅ Dependencies installed!")

## 2. Dataset Setup

Choose your dataset and download/prepare it.

In [ ]:
# Configuration
from pathlib import Path

DATASET = "cic_ids2018"  # Options: nsl_kdd, cic_ids2017, cic_ids2018, unsw_nb15, unified
DATA_ROOT = Path("/content/data")
DATA_ROOT.mkdir(exist_ok=True)

print(f"Selected dataset: {DATASET}")
print(f"Data root: {DATA_ROOT}")

In [ ]:
# Download CIC-IDS2018 from Kaggle (requires Kaggle API)
# Option 1: Using Kaggle API (recommended)
if DATASET == "cic_ids2018":
    print("To download CIC-IDS2018:")
    print("1. Install Kaggle: !pip install kaggle")
    print("2. Upload kaggle.json (from Kaggle account settings)")
    print("3. Run: !mkdir -p ~/.kaggle && cp kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json")
    print("4. Then run:")
    print(f"   !kaggle datasets download -d solarmainframe/ids-intrusion-csv -p {DATA_ROOT}")
    print(f"   !unzip -q {DATA_ROOT}/ids-intrusion-csv.zip -d {DATA_ROOT}/CIC-IDS2018")
    print("\n⚠️  Or manually upload dataset files via Colab file browser")

# Option 2: Manual upload via Colab file browser
# Files > Upload to session storage > Extract to DATA_ROOT

In [ ]:
# Preprocess the dataset (optional - training will preprocess if needed)
# Note: preprocess_data.py only supports: nsl_kdd, cic_ids2017, unified
# For cic_ids2018 and unsw_nb15, preprocessing happens automatically during training

import sys
import subprocess

# Only preprocess if dataset is supported by preprocess script
if DATASET in ["nsl_kdd", "cic_ids2017", "unified"]:
    print(f"Preprocessing {DATASET}...")
    
    cmd = [
        "python", "scripts/preprocess_data.py",
        "--data-root", str(DATA_ROOT),
        "--dataset", DATASET
    ]
    
    result = subprocess.run(cmd, cwd=repo_dir)
    
    if result.returncode == 0:
        print("✅ Preprocessing complete!")
    else:
        print(f"❌ Preprocessing failed with exit code {result.returncode}")
else:
    print(f"⚠️  Dataset {DATASET} will be preprocessed automatically during training")
    print("Skipping standalone preprocessing step...")

## 3. Training

Configure training parameters and start training.

In [ ]:
# Training configuration
EPOCHS = 100
BATCH_SIZE = 64  # Reduced for Colab memory (default is 256, use 64-128 for Colab)
MAX_SAMPLES = 500000  # Limit dataset size for Colab RAM (None = use all, 500k works well)
CHECKPOINT_INTERVAL = 10  # Save checkpoint every N epochs
SEED = 42
USE_AMP = True  # Mixed precision (faster on modern GPUs)
NUM_WORKERS = 2  # Reduce if you get errors

# Clear GPU memory before training
import gc
import torch
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    print(f"GPU memory cleared. Available: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

print(f"\nTraining configuration:")
print(f"  Dataset: {DATASET}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Max samples: {MAX_SAMPLES if MAX_SAMPLES else 'all'}")
print(f"  Epochs: {EPOCHS}")
print(f"  Mixed precision: {USE_AMP}")
print(f"  Seed: {SEED}")

In [ ]:
# Start training
import subprocess

cmd = [
    "python", "scripts/train_gan.py",
    "--data-root", str(DATA_ROOT),
    "--dataset", DATASET,
    "--epochs", str(EPOCHS),
    "--batch-size", str(BATCH_SIZE),
    "--checkpoint-interval", str(CHECKPOINT_INTERVAL),
    "--seed", str(SEED),
    "--num-workers", str(NUM_WORKERS),
]

if MAX_SAMPLES:
    cmd.extend(["--max-samples", str(MAX_SAMPLES)])

if USE_AMP:
    cmd.append("--amp")

print("Starting training...")
print(f"Command: {' '.join(cmd)}")
print("\n⚠️  This may take several hours. Checkpoints will be saved periodically.")
print("💡 If you get OOM errors, reduce MAX_SAMPLES (try 200000) or BATCH_SIZE (try 32)")

# Run training
result = subprocess.run(cmd, cwd=repo_dir)

if result.returncode == 0:
    print("\n✅ Training completed successfully!")
elif result.returncode == -9:
    print("\n❌ Training killed (exit code -9) - Out of Memory!")
    print("   Try reducing MAX_SAMPLES to 200000 or BATCH_SIZE to 32")
else:
    print(f"\n❌ Training failed with exit code {result.returncode}")

In [ ]:
# Check training outputs
output_files = [
    "training_stats.csv",
    "generator_best.pth",
    "generator_ema_best.pth",
    "generator.pth",
    "discriminator.pth",
]

print("Training outputs:")
for fname in output_files:
    path = repo_dir / fname
    if path.exists():
        size_mb = path.stat().st_size / 1e6
        print(f"  ✅ {fname} ({size_mb:.2f} MB)")
    else:
        print(f"  ❌ {fname} (not found)")

# List checkpoints
checkpoints = list(repo_dir.glob("checkpoint_epoch_*.pth"))
if checkpoints:
    print(f"\nCheckpoints found: {len(checkpoints)}")
    for cp in sorted(checkpoints)[-5:]:  # Show last 5
        size_mb = cp.stat().st_size / 1e6
        print(f"  {cp.name} ({size_mb:.2f} MB)")

## 4. Evaluation

Evaluate the trained generator with comprehensive metrics.

In [ ]:
# Evaluation configuration
GENERATOR_PATH = "generator_ema_best.pth"  # Use EMA version (better quality)
N_PER_CLASS = 2000  # Synthetic samples per class
TARGET_MINORITY = True  # Focus on minority classes
CV_FOLDS = 5  # Cross-validation folds
OUTPUT_DIR = "eval_outputs"

print(f"Evaluation configuration:")
print(f"  Generator: {GENERATOR_PATH}")
print(f"  Samples per class: {N_PER_CLASS}")
print(f"  Target minority: {TARGET_MINORITY}")
print(f"  CV folds: {CV_FOLDS}")

In [ ]:
# Run evaluation
import subprocess

cmd = [
    "python", "scripts/eval_gan.py",
    "--data-root", str(DATA_ROOT),
    "--dataset", DATASET,
    "--generator-path", GENERATOR_PATH,
    "--n-per-class", str(N_PER_CLASS),
    "--cv-folds", str(CV_FOLDS),
    "--output-dir", OUTPUT_DIR,
]

if TARGET_MINORITY:
    cmd.append("--target-minority")

print("Starting evaluation...")
print(f"Command: {' '.join(cmd)}")

result = subprocess.run(cmd, cwd=repo_dir)

if result.returncode == 0:
    print("\n✅ Evaluation completed successfully!")
else:
    print(f"\n❌ Evaluation failed with exit code {result.returncode}")

In [ ]:
# View evaluation results
import json

results_file = repo_dir / OUTPUT_DIR / "evaluation_results.json"
if results_file.exists():
    with open(results_file) as f:
        results = json.load(f)
    
    print("Evaluation Results Summary:")
    print("=" * 50)
    
    if "classifier_results" in results:
        print("\nClassifier Performance:")
        for classifier, metrics in results["classifier_results"].items():
            if "baseline" in metrics and "augmented" in metrics:
                baseline = metrics["baseline"]["mean"]
                augmented = metrics["augmented"]["mean"]
                improvement = augmented - baseline
                print(f"  {classifier}:")
                print(f"    Baseline F1: {baseline:.4f}")
                print(f"    Augmented F1: {augmented:.4f}")
                print(f"    Improvement: {improvement:+.4f}")
    
    if "distribution_metrics" in results:
        print("\nDistribution Quality:")
        for metric, value in results["distribution_metrics"].items():
            print(f"  {metric}: {value:.4f}")
else:
    print("⚠️  Results file not found. Run evaluation first.")

## 5. Visualizations

View generated plots and visualizations.

In [ ]:
# Display evaluation plots
from IPython.display import Image, display

plots_dir = repo_dir / OUTPUT_DIR / "plots"
if plots_dir.exists():
    plot_files = list(plots_dir.glob("*.png"))
    if plot_files:
        print(f"Found {len(plot_files)} plot(s):")
        for plot_file in plot_files:
            print(f"\n{plot_file.name}:")
            display(Image(str(plot_file)))
    else:
        print("No plots found in plots directory.")
else:
    print("Plots directory not found. Run evaluation first.")

## 6. Download Results

Download your trained models and results.

In [ ]:
# Create download package
import shutil
from pathlib import Path

download_dir = Path("/content/downloads")
download_dir.mkdir(exist_ok=True)

# Copy important files
files_to_download = [
    "generator_ema_best.pth",
    "generator_best.pth",
    "discriminator.pth",
    "training_stats.csv",
]

# Copy checkpoints
checkpoints = list(repo_dir.glob("checkpoint_epoch_*.pth"))
if checkpoints:
    checkpoint_dir = download_dir / "checkpoints"
    checkpoint_dir.mkdir(exist_ok=True)
    for cp in checkpoints:
        shutil.copy2(cp, checkpoint_dir / cp.name)
    print(f"Copied {len(checkpoints)} checkpoints")

# Copy evaluation outputs
eval_dir = repo_dir / OUTPUT_DIR
if eval_dir.exists():
    shutil.copytree(eval_dir, download_dir / OUTPUT_DIR, dirs_exist_ok=True)
    print("Copied evaluation outputs")

# Copy files
for fname in files_to_download:
    src = repo_dir / fname
    if src.exists():
        shutil.copy2(src, download_dir / fname)

print(f"\n✅ Files prepared for download in {download_dir}")
print("\nTo download:")
print("1. Use Colab file browser (left sidebar)")
print("2. Navigate to /content/downloads")
print("3. Right-click files and select 'Download'")

In [ ]:
# Alternative: Create a zip file for easy download
import shutil
from pathlib import Path
from google.colab import files

zip_path = "/content/stealthgan_results.zip"
download_dir = Path("/content/downloads")

if download_dir.exists():
    shutil.make_archive(
        zip_path.replace(".zip", ""),
        "zip",
        download_dir
    )
    
    size_mb = Path(zip_path).stat().st_size / 1e6
    print(f"✅ Created zip file: {zip_path} ({size_mb:.2f} MB)")
    print("\nDownloading zip file...")
    files.download(zip_path)
else:
    print("⚠️  Download directory not found. Run the previous cell first.")

## 7. Resume Training (Optional)

Resume training from a checkpoint if your session disconnects.

In [ ]:
# Resume training from checkpoint
CHECKPOINT_PATH = "checkpoint_epoch_50.pth"  # ⚠️ Update with your checkpoint name
RESUME_EPOCHS = 100  # Total epochs (will continue from checkpoint)

cmd = [
    "python", "scripts/train_gan.py",
    "--data-root", str(DATA_ROOT),
    "--dataset", DATASET,
    "--epochs", str(RESUME_EPOCHS),
    "--batch-size", str(BATCH_SIZE),
    "--resume", CHECKPOINT_PATH,
    "--seed", str(SEED),
    "--num-workers", str(NUM_WORKERS),
]

if MAX_SAMPLES:
    cmd.extend(["--max-samples", str(MAX_SAMPLES)])

if USE_AMP:
    cmd.append("--amp")

print("To resume training, update CHECKPOINT_PATH above and uncomment the last line:")
print(f"{' '.join(cmd)}")

# Uncomment to run:
# result = subprocess.run(cmd, cwd=repo_dir)

## Troubleshooting

**Common Issues:**

1. **Out of Memory (exit code -9)**:
   - Reduce `MAX_SAMPLES` (try 200000 or 100000)
   - Reduce `BATCH_SIZE` (try 32 or 16)
   - Use a smaller dataset (`nsl_kdd` instead of `cic_ids2018`)
2. **Session Timeout**: Colab free tier has 12hr limit. Use checkpoints to resume.
3. **Dataset Not Found**: Ensure dataset is downloaded and extracted correctly.
4. **Slow Training**: Enable GPU (Runtime > Change runtime type > GPU)

**Memory Guide for Colab Free Tier (12GB RAM):**
| Dataset | Recommended MAX_SAMPLES |
|---------|------------------------|
| NSL-KDD | None (all ~125K) |
| CIC-IDS2017 | 500000 |
| CIC-IDS2018 | 500000 |
| UNSW-NB15 | None (all ~175K) |

**Tips:**
- Save checkpoints frequently
- Use Colab Pro for longer sessions (24hr limit) and more RAM
- Download results before session expires
- Monitor GPU usage: `!nvidia-smi`

In [ ]:
# Monitor GPU usage
!nvidia-smi